In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os #operation system interface
import sys
import json
import uuid as uuid #universally unique identifiers (UUIDs)
import cv2
import re
import shutil
from operator import itemgetter
import math
import random

import shapely
import shapely.geometry
from shapely.geometry import Polygon,MultiPolygon
from shapely.validation import make_valid

In [21]:
# read in image file 
img=cv2.imread("/Users/lovely_shufan/Documents/REU_2023_Arline/raw_data/AMFinderImages/CNN2_Keyence_Nbenth_myc_8.jpg")
#cv2.imshow("img", img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#cv2.waitKey(1)

#img2=cv2.imread("/Users/lovely_shufan/Documents/REU_2023_Arline/AMFinderImages/CNN2_Keyence_Nbenth_myc_9.jpg")


In [22]:
# measure image dimensions
height = img.shape[0]
width = img.shape[1]

print(height, width)
#print(img2.shape[0],img2.shape[1])

# padd image to optimal input image dimension of 800 1333
    # padding to the left and the bottom of the image as it was mostly empty
pad_bot = height % 800 
pad_lft = width % 1333

print(pad_bot, pad_lft)

3204 9560
4 229


In [23]:
# padding image 
img_pad = cv2.copyMakeBorder(img,0,pad_bot,pad_lft,0,cv2.BORDER_CONSTANT,value=[0,0,100])
#cv2.imshow("img", img)
#cv2.imshow("img_padded",img_pad)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#cv2.waitKey(1)

In [24]:
tiles = []
tiles.append([0,1,2,3])
print(tiles)
tiles.append([5,6,7,8])
print(tiles)

[[0, 1, 2, 3]]
[[0, 1, 2, 3], [5, 6, 7, 8]]


In [65]:
# FUNCTION tileImage(img)
#    '''
#    :param img:
#    :return list of coordinates:
#    :rtype list:
#    Objective: output a list of bouding coordinates of the tiles   '''

tiles = [] # an empty list 
for i in range(0,img_pad.shape[0],800):
    for j in range(0,img_pad.shape[1],1333):
        #print(i, " ", j)
        xmin = j
        xmax = j + 1333
        ymin = i
        ymax =i + 800
        tiles.append([xmin,ymin,xmax,ymax])
print(tiles)

[[0, 0, 1333, 800], [1333, 0, 2666, 800], [2666, 0, 3999, 800], [3999, 0, 5332, 800], [5332, 0, 6665, 800], [6665, 0, 7998, 800], [7998, 0, 9331, 800], [9331, 0, 10664, 800], [0, 800, 1333, 1600], [1333, 800, 2666, 1600], [2666, 800, 3999, 1600], [3999, 800, 5332, 1600], [5332, 800, 6665, 1600], [6665, 800, 7998, 1600], [7998, 800, 9331, 1600], [9331, 800, 10664, 1600], [0, 1600, 1333, 2400], [1333, 1600, 2666, 2400], [2666, 1600, 3999, 2400], [3999, 1600, 5332, 2400], [5332, 1600, 6665, 2400], [6665, 1600, 7998, 2400], [7998, 1600, 9331, 2400], [9331, 1600, 10664, 2400], [0, 2400, 1333, 3200], [1333, 2400, 2666, 3200], [2666, 2400, 3999, 3200], [3999, 2400, 5332, 3200], [5332, 2400, 6665, 3200], [6665, 2400, 7998, 3200], [7998, 2400, 9331, 3200], [9331, 2400, 10664, 3200], [0, 3200, 1333, 4000], [1333, 3200, 2666, 4000], [2666, 3200, 3999, 4000], [3999, 3200, 5332, 4000], [5332, 3200, 6665, 4000], [6665, 3200, 7998, 4000], [7998, 3200, 9331, 4000], [9331, 3200, 10664, 4000]]


In [26]:
# get the coordinates of tiles
    # knowing the coordinates is sufficient for tilling
for ind,tile in enumerate(tiles):
    print(ind,tile)
    

0 [0, 0, 1333, 800]
1 [1333, 0, 2666, 800]
2 [2666, 0, 3999, 800]
3 [3999, 0, 5332, 800]
4 [5332, 0, 6665, 800]
5 [6665, 0, 7998, 800]
6 [7998, 0, 9331, 800]
7 [9331, 0, 10664, 800]
8 [0, 800, 1333, 1600]
9 [1333, 800, 2666, 1600]
10 [2666, 800, 3999, 1600]
11 [3999, 800, 5332, 1600]
12 [5332, 800, 6665, 1600]
13 [6665, 800, 7998, 1600]
14 [7998, 800, 9331, 1600]
15 [9331, 800, 10664, 1600]
16 [0, 1600, 1333, 2400]
17 [1333, 1600, 2666, 2400]
18 [2666, 1600, 3999, 2400]
19 [3999, 1600, 5332, 2400]
20 [5332, 1600, 6665, 2400]
21 [6665, 1600, 7998, 2400]
22 [7998, 1600, 9331, 2400]
23 [9331, 1600, 10664, 2400]
24 [0, 2400, 1333, 3200]
25 [1333, 2400, 2666, 3200]
26 [2666, 2400, 3999, 3200]
27 [3999, 2400, 5332, 3200]
28 [5332, 2400, 6665, 3200]
29 [6665, 2400, 7998, 3200]
30 [7998, 2400, 9331, 3200]
31 [9331, 2400, 10664, 3200]
32 [0, 3200, 1333, 4000]
33 [1333, 3200, 2666, 4000]
34 [2666, 3200, 3999, 4000]
35 [3999, 3200, 5332, 4000]
36 [5332, 3200, 6665, 4000]
37 [6665, 3200, 7998, 400

In [37]:
# read in the coordinates of annotations
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
annotab = pd.read_csv("/Users/lovely_shufan/Documents/REU_2023_Arline/raw_data/CSVannotations/TrainingSet003_Arline_annotation_via_project_20July23_12.28pm_csv.csv", delimiter = ",")
files = annotab['filename'].unique()
#print_full(annotab.iloc[:, [5]])
#print_full(annotab['filename'])

# drop template images
exclude_file = ['Snap-12327-Image Export-127.jpg', 'Snap-12291-Image Export-91.jpg','Snap-12220-Image Export-20.jpg']
annotab = annotab[~annotab['filename'].isin(exclude_file)]
#print(image_names)
#print(annotab['filename'].unique())

# drop empty and non arb annotations
#print(annotab['region_attributes'].unique())
exclude_anno = ['{"object":"AMF internal hypha"}', '{}']
annotab = annotab[~annotab['region_attributes'].isin(exclude_anno)]

# drop shapes with less than three points
exclude_line_shapes = []
for i in range(0,annotab.shape[0],1):
    anno = json.loads(annotab.iloc[i,]["region_shape_attributes"])
    if len(anno['all_points_x']) < 3:
        exclude_line_shapes.append(i)
#print(exclude_line_shapes) # there is no shape with less than three points

files = annotab['filename'].unique()
file = 'CNN2_Keyence_Nbenth_myc_8.jpg'
#print(file)

CNN2_Keyence_Nbenth_myc_8.jpg


In [28]:
# parse annotation label
classes = ['AMF arbuscule island', 'AMF arbsucule forest']
subtab = annotab[annotab['filename'] == 'CNN2_Keyence_Nbenth_myc_8.jpg']
#print(subtab.shape[0]) # df.shape is getting the dimension of the dataframe, [0] is the #row
loadcldict = json.loads(subtab.iloc[1,:]['region_attributes'])
category_id = classes.index(loadcldict['object'])
#print(loadcldict)

# parse annotation shape coordinates
anno = json.loads(subtab.iloc[0,]["region_shape_attributes"])
print(anno)
#len(anno['all_points_x']) == len(anno['all_points_y'])

{'name': 'polygon', 'all_points_x': [3214, 3209, 3206, 3203, 3209, 3217, 3224, 3228, 3233, 3239, 3244, 3248, 3249, 3251, 3251, 3236, 3235, 3224, 3218], 'all_points_y': [246, 249, 249, 251, 257, 264, 270, 271, 273, 275, 276, 274, 270, 264, 258, 247, 245, 244, 241]}


In [67]:
# FUNCTION convertPoints(anno)
    #'''
    #:param anno: dictionary format of region_shape_attributes
    #:i.e.: {'name': 'polygon', 'all_points_x': [3242, 3235, 3234, 3233, 3232, 3233, 3240, 3241, 3243, 3244, 3245, 3249, 3249, 3251, 3256, 3260, 3263, 3268, 3268, 3267, 3265, 3264], 'all_points_y': [295, 295, 303, 306, 312, 318, 323, 323, 324, 327, 328, 329, 335, 339, 341, 342, 342, 340, 335, 332, 326, 325]}
    #:return points: list of points in pairs
    #'''
points = []
                    
for i in range(0,len(anno["all_points_x"]),1):
    points.append([anno["all_points_x"][i],anno["all_points_y"][i]])
    
print(points)


[[1787, 1070], [1777, 1060], [1774, 1055], [1769, 1055], [1764, 1056], [1762, 1059], [1768, 1068], [1774, 1074], [1778, 1076], [1783, 1077]]


In [69]:
print(len(tiles))
tile = tiles[0]
print(type(tile[0]))
print(type(points))

40
<class 'int'>
<class 'list'>


In [66]:
# FUNCTION intersectBoundingBox(points, xmin, xmax, ymin, ymax)
    #'''
    #:param points: list of point pairs
    #:i.e.: [[3242, 295], [3235, 295], [3234, 303]...]
    #:return converted_points: list of point pairs of which point coordinates are converted from image to tile
    #'''
for tile in tiles:
    print(tile)
    xmin = tile[0]
    xmax = tile[2]
    ymin = tile[1]
    ymax = tile[3]
    converted_points = []
    for ind, p in enumerate(points):
        p2 = list(p)
        if p[0]>=xmin and p[0]<=xmax and p[1]>=ymin and p[1]<=ymax:
            p2[0]=p[0]-xmin
            p2[1]=p[1]-ymin
            converted_points.append(p2)
            print(p)
            print(p2)
            
        #print(converted_points)

[0, 0, 1333, 800]
[1333, 0, 2666, 800]
[2666, 0, 3999, 800]
[3214, 246]
[548, 246]
[3209, 249]
[543, 249]
[3206, 249]
[540, 249]
[3203, 251]
[537, 251]
[3209, 257]
[543, 257]
[3217, 264]
[551, 264]
[3224, 270]
[558, 270]
[3228, 271]
[562, 271]
[3233, 273]
[567, 273]
[3239, 275]
[573, 275]
[3244, 276]
[578, 276]
[3248, 274]
[582, 274]
[3249, 270]
[583, 270]
[3251, 264]
[585, 264]
[3251, 258]
[585, 258]
[3236, 247]
[570, 247]
[3235, 245]
[569, 245]
[3224, 244]
[558, 244]
[3218, 241]
[552, 241]
[3999, 0, 5332, 800]
[5332, 0, 6665, 800]
[6665, 0, 7998, 800]
[7998, 0, 9331, 800]
[9331, 0, 10664, 800]
[0, 800, 1333, 1600]
[1333, 800, 2666, 1600]
[2666, 800, 3999, 1600]
[3999, 800, 5332, 1600]
[5332, 800, 6665, 1600]
[6665, 800, 7998, 1600]
[7998, 800, 9331, 1600]
[9331, 800, 10664, 1600]
[0, 1600, 1333, 2400]
[1333, 1600, 2666, 2400]
[2666, 1600, 3999, 2400]
[3999, 1600, 5332, 2400]
[5332, 1600, 6665, 2400]
[6665, 1600, 7998, 2400]
[7998, 1600, 9331, 2400]
[9331, 1600, 10664, 2400]
[0, 2400,

In [35]:
p1 = Polygon([(0,0), (1,1), (1,0)])
p2 = Polygon([(0,1), (1,0), (1,1)])
p1.intersects(p2)
print(type(img))
print(tiles)
print(files)

<class 'numpy.ndarray'>
[[0, 0, 1333, 800], [1333, 0, 2666, 800], [2666, 0, 3999, 800], [3999, 0, 5332, 800], [5332, 0, 6665, 800], [6665, 0, 7998, 800], [7998, 0, 9331, 800], [9331, 0, 10664, 800], [0, 800, 1333, 1600], [1333, 800, 2666, 1600], [2666, 800, 3999, 1600], [3999, 800, 5332, 1600], [5332, 800, 6665, 1600], [6665, 800, 7998, 1600], [7998, 800, 9331, 1600], [9331, 800, 10664, 1600], [0, 1600, 1333, 2400], [1333, 1600, 2666, 2400], [2666, 1600, 3999, 2400], [3999, 1600, 5332, 2400], [5332, 1600, 6665, 2400], [6665, 1600, 7998, 2400], [7998, 1600, 9331, 2400], [9331, 1600, 10664, 2400], [0, 2400, 1333, 3200], [1333, 2400, 2666, 3200], [2666, 2400, 3999, 3200], [3999, 2400, 5332, 3200], [5332, 2400, 6665, 3200], [6665, 2400, 7998, 3200], [7998, 2400, 9331, 3200], [9331, 2400, 10664, 3200], [0, 3200, 1333, 4000], [1333, 3200, 2666, 4000], [2666, 3200, 3999, 4000], [3999, 3200, 5332, 4000], [5332, 3200, 6665, 4000], [6665, 3200, 7998, 4000], [7998, 3200, 9331, 4000], [9331, 3200,

In [42]:
print(file[:-4]+'_'+str(tile[0])+'_'+str(tile[1])+'_'+str(tile[2])+'_'+str(tile[3]))

CNN2_Keyence_Nbenth_myc_8_0_0_1333_800


In [43]:
tile_id = file[:-4] + '_'+ str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
record ={}
record["filename"] = tile_id + ".jpg"
record["height"] = 800
record["width"] = 1333
print(record)

{'filename': 'CNN2_Keyence_Nbenth_myc_8_9331_3200_10664_4000.jpg', 'height': 800, 'width': 1333}


In [52]:
for col in annotab.columns:
    print(col)
print(annotab[['filename','region_id']])

print(uuid.uuid4())

filename
file_size
file_attributes
region_count
region_id
region_shape_attributes
region_attributes
                            filename  region_id
17     CNN2_Keyence_Nbenth_myc_8.jpg          0
18     CNN2_Keyence_Nbenth_myc_8.jpg          1
19     CNN2_Keyence_Nbenth_myc_8.jpg          2
20     CNN2_Keyence_Nbenth_myc_8.jpg          3
21     CNN2_Keyence_Nbenth_myc_8.jpg          4
...                              ...        ...
7943  CNN2_Keyence_Nbenth_myc_20.jpg        440
7944  CNN2_Keyence_Nbenth_myc_20.jpg        441
7945  CNN2_Keyence_Nbenth_myc_20.jpg        442
7946  CNN2_Keyence_Nbenth_myc_20.jpg        443
7947  CNN2_Keyence_Nbenth_myc_20.jpg        444

[7924 rows x 2 columns]
44c9b2ce-26ef-4681-9706-94cbb7a1bd77


In [56]:
tab_rec = subtab.iloc[0]
loadcldict = json.loads(tab_rec['region_attributes'])
print(tab_rec)
print(loadcldict)
print(loadcldict['object'])
category_id=classes.index(loadcldict['object'])
print(category_id)

filename                                       CNN2_Keyence_Nbenth_myc_8.jpg
file_size                                                           16866163
file_attributes                                                           {}
region_count                                                             723
region_id                                                                  0
region_shape_attributes    {"name":"polygon","all_points_x":[3214,3209,32...
region_attributes                          {"object":"AMF arbuscule island"}
Name: 17, dtype: object
{'object': 'AMF arbuscule island'}
AMF arbuscule island
0


In [ ]:
# to check whether all the annotations are valid after make valid
validlist = []
for i in range(0,annotab.shape[0],1):
    anno = json.loads(annotab.iloc[i]["region_shape_attributes"])
    #print(anno)
    points = convertPoints(anno)
    #print(points)
    polygon_anno = Polygon(points)
    polygon_anno = make_valid(polygon_anno)
    validlist.append(polygon_anno.is_valid)
print(sum(validlist))
print(annotab.shape[0])

In [ ]:
# test FUNCTION intersectmask
subtab = annotab[annotab['filename'] == 'CNN2_Keyence_Nbenth_myc_9.jpg']
img=readImage(img_dir,'CNN2_Keyence_Nbenth_myc_9.jpg')
tiles=tileImage(img)

for ind,tile in enumerate(tiles):
    print(tile)
   
    # get the coordinate over the tile image
    xmin=tile[0]
    ymin=tile[1]
    xmax=tile[2]
    ymax=tile[3]
    tilebox = Polygon([(xmin,ymin),(xmax,ymin),(xmax,ymax),(xmin,ymax)])
    
    #create a record for each intersection
    ## the tile record
    record = {}    # empty dictionary
    record["filename"] = 'CNN2_Keyence_Nbenth_myc_9' + '_'+ str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax) + '.jpg'
    record["height"] = 800
    record["width"] = 1333
    for i in range(0,subtab.shape[0],1):
        anno = json.loads(subtab.iloc[i]["region_shape_attributes"])
        points = convertPoints(anno)
        polygon_points = Polygon(points)
        polygon_points = make_valid(polygon_points)
        
        # create intersection between tile box and annotation
        intersect_anno = polygon_points.intersection(tilebox)
        intersect_points = []
        if not intersect_anno.is_empty: #empty intersections are dropped
            #print(i)
            is_polygon = intersect_anno.geom_type=='Polygon'
            is_multipolygon = intersect_anno.geom_type=='MultiPolygon'
            is_mutliothers = intersect_anno.geom_type=='GeometryCollection'
            
            
            if is_polygon or is_multipolygon or is_mutliothers: # points and lines are dropped
                
                if is_polygon:
                    #print('Polygon')
                    # convert new annotations from polygons to points
                    exterior = list(intersect_anno.exterior.coords)[:-1] #remove the last point which is the same as the first point
                    interior = list(intersect_anno.interiors)
                
                    if interior != []:
                        # merge interior point/linearring with exterior points
                        interior = GeometryCollection(interior)
                        for geom in interior.geoms: 
                            for i in mapping(geom)['coordinates'][:-1]:
                                exterior.append(i)
                        new_anno = list(exterior)
                        intersect_points = list(new_anno)
                        print(intersect_points)
                    else: # if intersected annotation is polygon without holes
                        intersect_points = list(exterior)
                else:
                    if is_multipolygon: #remove the small polygons and keep the largest
                        #print('MultiPolygon')
                        # keep the largest polygon
                        poly_area = []
                        for geom in intersect_anno.geoms:
                            poly_area.append(geom.area)
                        new_anno = intersect_anno.geoms[poly_area.index(max(poly_area))]
                        new_anno = list(new_anno.exterior.coords)[:-1]
                        intersect_points = list(new_anno)
                        print(intersect_points)
                        
                    else: #is geometrycollection
                        #print('GeometryCollection')
                        # remove linestring
                        for geom in points2.geoms:
                            if geom.geom_type == 'Point' or geom.geom_type == 'LineString':
                                continue
                            else:
                                new_anno = geom
                        
                        # new_anno should be a polygon with or without holes
                        exterior = list(new_anno.exterior.coords)[:-1] #remove the last point which is the same as the first point
                        interior = list(new_anno.interiors)
                        if interior != []:                        
                            # merge interior point/linearring with exterior points
                            interior = GeometryCollection(interior)

                            for geom in interior.geoms:
                                for i in mapping(geom)['coordinates'][:-1]: #remove duplicated points
                                    exterior.append(i)
                            new_anno = list(exterior)
                            intersect_points = list(new_anno)
                            print(intersect_points)
                        else:
                            new_anno = list(new_anno.exterior.coords)[:-1]
                            intersect_points = list(new_anno)
                            print(intersect_points)
                        transformed_intersect_points = transformPoints(intersect_points)
        #print(intersect_points)    
            #for ind, point in enumerate(new_anno):
                #print(ind, point)
                    #p2=list(p)
                    #p2[0]=p2[0]-xmin
                    #p2[1]=p2[1]-ymin
                    #converted_points.append(p2)
            

        
        

In [ ]:
# FUNCTION IntersectSegmentations\

#def IntersectSegmentations(img_dir,output_dir,tiles, img, annotab, file,classes):
#    '''
#   :param tiles:
#    :paramtype list: list of tile corner coordinates
#    :param img:
#    :paramtype numpy array: padded image
#    :param annotab:
#    :paramtype pandas dataframe:
#    :param files:
#    :paramtype list: list of image names
#    :param classes:
#    :paramtype list:
#    :return dataset_dicts:
#    :rtype list:
#    Objective: iterate over each of the annotations in the image and intersect them with the tile bounding boxes
#    '''
    #filename=os.path.join(img_dir,file)
    #records=[]
    #nonvalid_seg_counter=0
    
    # iterate over the tile coordinates
    for ind,tile in enumerate(tiles):
        record = {} # an empty dictionary
        # get the coordinate over the tile image
        xmin = tile[0]
        ymin = tile[1]
        xmax = tile[2]
        ymax = tile[3]
        
        # make a tile id using the xmin,ymin,xmax,ymax and the filename
        tile_id = file[:-4] + '_'+ str(xmin)+"_"+str(ymin)+"_"+str(xmax)+"_"+str(ymax)
        
        # begin building the record by adding the information for the COCO dataset
        record["filename"] = tile_id + '.jpg'
        record["height"] = 800
        record["width"] = 1333
        
        # make an empty list of objects for record annotation
        record["annotations"] = []
        subtab = annotab[annotab['filename'] == file]
        
        objs =[]
        for anno_i in range(subtab.shape[0]): #iterate over each annotation
            # make a UID for each polygon
            uid = str(uuid.uuid4())
            
            # get the catagory id
            tab_rec=subtab.iloc[anno_i]
            loadcldict=json.loads(tab_rec['region_attributes'])
            category_id=classes.index(loadcldict['object'])
            
            # convert the category id to the class name by using the classes array
            #className=classes[category_id]
            
            anno=json.loads(tab_rec["region_shape_attributes"])
            #if len(anno)==0:
            #    continue
            
            #print(anno)
            points=convertPoints(anno)
            # quick check 1 (this was for speed purpose but the crossing patterns are more complex and might have no point within)
            # converted_points_in=intersectBoundingBox(points,xmin,ymin,xmax,ymax)
            converted_points_in=[1]
            if len(converted_points_in) >= 1:
                converted_points_list=intersectmask(points,xmin,ymin,xmax,ymax)
                
                for indp,converted_points in enumerate(converted_points_list):
                    if len(converted_points)>0:
                        Sxmin=min(converted_points,key=lambda x:x[0])[0]
                        Symin=min(converted_points,key=lambda x:x[1])[1]
                        Sxmax=max(converted_points,key=lambda x:x[0])[0]
                        Symax=max(converted_points,key=lambda x:x[1])[1]
                        converted_points=[item for sublist in converted_points for item in sublist]
                        Segbbox = [Sxmin,Symin,Sxmax,Symax]
                        obj = {
                            'filename': tile_id + '.jpg',
                            "image_id": tile_id,
                            'structure_id': uid,
                            'height': 512,
                            'width': 512,
                            "category_id": className,
                            "bbox": Segbbox,
                            "segmentation": converted_points,
                            "bbox_mode": 'BoxMode.XYXY_ABS',
                            "iscrowd":0,
                            }
                        #objs.append(obj)
                        record["annotations"].append(obj)
                    else:
                        print('2:'+str(ind)+'_'+str(anno_i)+'_'+className)
                        nonvalid_seg_counter=nonvalid_seg_counter+1
        if len(record['annotations']) > 0:
            #subset the image to the tile coordinates
            subimg=img[ymin:ymax,xmin:xmax]
            # write the tile image to the output directory
            cv2.imwrite(os.path.join(output_dir,tile_id+'.jpg'),subimg)
            records.append(record)
    print('nonvalid segmentation: '+str(nonvalid_seg_counter))
    return records